In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
#Initializing the dataset paths
path ='/content/drive/MyDrive/ALDA_project/Training_dataset/'
testing_path = '/content/drive/MyDrive/ALDA_project/test_set/'
img_path= path+"devset_images/"
label_path = path+ 'devset_images_gt.csv'
test_img_path= testing_path+"testset_images/"
test_label_path = testing_path+ 'testset_images_gt.csv'

In [ ]:
#Formatting the input images
def append_ext(fn):
    return fn+".jpg"
traindf=pd.read_csv(label_path,dtype=str)
traindf.columns=(['id', 'label'])
traindf["id"]=traindf["id"].apply(append_ext)
testdf=pd.read_csv(test_label_path,dtype=str)
testdf.columns=(['id', 'label'])
testdf["id"]=testdf["id"].apply(append_ext)
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.10)
test_datagen = ImageDataGenerator(rescale=1./255.)

In [ ]:
#Creating the dataset generators for train, validation and test
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=img_path,
x_col="id",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode='categorical',
target_size=(224,224))
valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=img_path,
x_col="id",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))
test_generator=datagen.flow_from_dataframe(
dataframe=testdf,
directory=test_img_path,
x_col="id",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 15 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 4738 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 15 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 526 validated image filenames belonging to 2 classes.
Found 1317 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
#Initiazling the image dimensions
img_height = 224
img_width = 224
IMG_SIZE = (img_height, img_width)

In [ ]:
#Defining the pre-trained model

preprocess_input = tf.keras.applications.inception_v3.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255., offset= -1)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
image_batch, label_batch = next(iter(train_generator))
feature_batch = base_model(image_batch)
print(feature_batch.shape)
print(label_batch.shape)
base_model.trainable = False

87924736/87910968 [==============================] - 2s 0us/step
(32, 5, 5, 2048)
(32, 2)


In [ ]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 2048)


In [ ]:
prediction_layer = tf.keras.layers.Dense(2)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 2)


In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0     

In [ ]:
#Training of the model
initial_epochs = 30
history = model.fit(train_generator,
                    epochs=initial_epochs,
                    validation_data=valid_generator)

Epoch 1/30
149/149 [==============================] - 2374s 16s/step - loss: 0.6538 - accuracy: 0.6271 - val_loss: 0.6283 - val_accuracy: 0.6559
Epoch 2/30
149/149 [==============================] - 46s 309ms/step - loss: 0.6394 - accuracy: 0.6296 - val_loss: 0.6160 - val_accuracy: 0.6749
Epoch 3/30
149/149 [==============================] - 46s 307ms/step - loss: 0.6266 - accuracy: 0.6437 - val_loss: 0.6063 - val_accuracy: 0.6920
Epoch 4/30
149/149 [==============================] - 45s 304ms/step - loss: 0.6224 - accuracy: 0.6456 - val_loss: 0.5924 - val_accuracy: 0.6844
Epoch 5/30
149/149 [==============================] - 45s 303ms/step - loss: 0.6233 - accuracy: 0.6463 - val_loss: 0.5929 - val_accuracy: 0.6958
Epoch 6/30
 13/149 [=>............................] - ETA: 37s - loss: 0.6352 - accuracy: 0.6298

KeyboardInterrupt: ignored

In [ ]:
#Fine-tuning the model
base_model.trainable = True

print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])
model.summary()

Number of layers in the base model:  311
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pool

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
fine_tune_epochs = 5
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_generator,
                         epochs=total_epochs,
                         initial_epoch=10,
                         validation_data=valid_generator)

Epoch 11/35
149/149 [==============================] - 80s 444ms/step - loss: 0.6218 - accuracy: 0.6568 - val_loss: 0.5631 - val_accuracy: 0.7091
Epoch 12/35
149/149 [==============================] - 64s 429ms/step - loss: 0.5906 - accuracy: 0.6843 - val_loss: 0.5386 - val_accuracy: 0.7357
Epoch 13/35
149/149 [==============================] - 64s 428ms/step - loss: 0.5815 - accuracy: 0.6881 - val_loss: 0.5542 - val_accuracy: 0.7034
Epoch 14/35
149/149 [==============================] - 64s 429ms/step - loss: 0.5740 - accuracy: 0.6948 - val_loss: 0.5295 - val_accuracy: 0.7338
Epoch 15/35
149/149 [==============================] - 64s 429ms/step - loss: 0.5616 - accuracy: 0.6971 - val_loss: 0.5339 - val_accuracy: 0.7510
Epoch 16/35
149/149 [==============================] - 64s 429ms/step - loss: 0.5521 - accuracy: 0.7033 - val_loss: 0.5209 - val_accuracy: 0.7414
Epoch 17/35
149/149 [==============================] - 64s 430ms/step - loss: 0.5470 - accuracy: 0.7138 - val_loss: 0.5414 -

In [ ]:
#Generating the predictions for test dataset
test_generator.reset()
y_pred = model.predict_generator(test_generator, verbose = True)
y_pred = np.argmax(y_pred, axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


42/42 [==============================] - 13s 274ms/step


In [ ]:
y_true = test_generator.classes

In [ ]:
#Evaluation metrics for the test dataset
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.73      0.68       839
           1       0.36      0.26      0.30       478

    accuracy                           0.56      1317
   macro avg       0.50      0.50      0.49      1317
weighted avg       0.53      0.56      0.54      1317

